In [ ]:
from src.utils import visualisations
from src.utils.utils import DataTransformer
from torch_geometric.data import Data
from src.datasets.zinc_smiles_generation import ZincSmiles
from src.datasets.zinc_pairs_v2 import ZincPairsV2
from src.encoding.decoder import is_induced_subgraph_by_features


train_ds = ZincPairsV2(split="train", base_dataset=ZincSmiles(split="train"), dev=True)
test_ds = ZincPairsV2(split="test", base_dataset=ZincSmiles(split="train"), dev=True)
valid_ds = ZincPairsV2(split="valid", base_dataset=ZincSmiles(split="train"), dev=True)
counter = 1
for item in [*train_ds, *test_ds, *valid_ds]:
    g1 = Data(x=item.x1, edge_index=item.edge_index1)
    g2 = Data(x=item.x2, edge_index=item.edge_index2)
    y = bool(item.y.item())

    g1_x = DataTransformer.pyg_to_nx(g1)
    g2_x = DataTransformer.pyg_to_nx(g2)
    is_induced = is_induced_subgraph_by_features(g1_x, g2_x, node_keys=["feat"])

    if is_induced != y:
        print(f"[{counter}] WRONG")
        visualisations.draw_nx_with_atom_colorings(g1_x, label=f"[{counter}]Actual: {is_induced}, Y: {y}")
        visualisations.draw_nx_with_atom_colorings(g2_x, label=f"[{counter}]Actual: {is_induced}, Y: {y}")
    else:
        print(f"[{counter}] OK")
    counter += 1